In [1]:
import pandas as pd
import os
import gzip
import json



In [2]:
def load_json_gz_file(filepath):
    with gzip.open(filepath, 'rt', encoding='utf-8') as file:
        print(filepath)
        data = [json.loads(line) for line in file]
    return data
def concatenate_json_files(directory):
    all_data = []
    for filename in os.listdir(directory):
        if filename.endswith(".json.gz"):
            filepath = os.path.join(directory, filename)
            file_data = load_json_gz_file(filepath)
            all_data.extend(file_data)
    
    df = pd.DataFrame(all_data)
    return df

In [ ]:
#  directory containing the JSON files
directory = './genre'

# Concatenating all JSON files into a single DataFrame
df = concatenate_json_files(directory)

### This method took a LOT of time so we had to process by chunks and save multiple dataframes that will later merged and cleaned ...

In [ ]:
import pandas as pd
import os
import gzip
import json

def process_json_gz_file(filepath, chunk_size=10000):
    with gzip.open(filepath, 'rt', encoding='utf-8') as file:
        chunk = []
        for i, line in enumerate(file):
            chunk.append(json.loads(line))
            if (i + 1) % chunk_size == 0:
                yield pd.DataFrame(chunk)
                chunk = []
        if chunk:
            yield pd.DataFrame(chunk)

def save_intermediate_csv_files(directory, output_prefix, chunk_size=10000, batch_size=10):
    batch_data = []
    batch_count = 0
    
    for filename in os.listdir(directory):
        if filename.endswith(".json.gz"):
            filepath = os.path.join(directory, filename)
            for chunk_df in process_json_gz_file(filepath, chunk_size):
                batch_data.append(chunk_df)
                
                if len(batch_data) >= batch_size:
                    combined_df = pd.concat(batch_data, ignore_index=True)
                    combined_df.to_csv(f"{output_prefix}_{batch_count}.csv", index=False)
                    batch_data = []
                    batch_count += 1

    # Saving any remaining data
    if batch_data:
        combined_df = pd.concat(batch_data, ignore_index=True)
        combined_df.to_csv(f"{output_prefix}_{batch_count}.csv", index=False)


C:\Users\Sara\AppData\Local\Temp\ipykernel_10556\3470257456.py:42: DtypeWarning: Columns (7,29,30,31,33,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  all_dfs.append(pd.read_csv(filename))
C:\Users\Sara\AppData\Local\Temp\ipykernel_10556\3470257456.py:42: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  all_dfs.append(pd.read_csv(filename))
C:\Users\Sara\AppData\Local\Temp\ipykernel_10556\3470257456.py:42: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  all_dfs.append(pd.read_csv(filename))


In [3]:
10000*14

140000

In [ ]:
def concatenate_all_csv_files(output_prefix, final_output_file):
    all_dfs = []
    for filename in os.listdir('.'):
        if filename.startswith(output_prefix) and filename.endswith(".csv"):
            all_dfs.append(pd.read_csv(filename))
    
    final_df = pd.concat(all_dfs, ignore_index=True)
    final_df.to_csv(final_output_file, index=False)



# Intermediate and final output file prefixes
output_prefix = 'intermediate_goodreads_data'
final_output_file = 'final_concatenated_goodreads_data.csv'


# Concatenating all intermediate CSV files into one final CSV file
concatenate_all_csv_files(output_prefix, final_output_file)

In [2]:
## Finally this was just for testing purposes since the csv were heavy we tried to take just 2000 of each at the end from 14 csv we just took 6

def concatenate_all_csv_files(output_prefix, final_output_file):
    all_dfs = []
    for filename in os.listdir('.'):
        if filename.startswith(output_prefix) and filename.endswith(".csv"):
            all_dfs.append(pd.read_csv(filename).head(2000))
    
    final_df = pd.concat(all_dfs, ignore_index=True)
    final_df.to_csv(final_output_file, index=False)


output_prefix = 'intermediate_goodreads_data'
final_output_file = 'final_concatenated_goodreads_data_small.csv'

concatenate_all_csv_files(output_prefix, final_output_file)

C:\Users\Sara\AppData\Local\Temp\ipykernel_1852\3516056252.py:5: DtypeWarning: Columns (7,29,30,31,33,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  all_dfs.append(pd.read_csv(filename).head(2000))
C:\Users\Sara\AppData\Local\Temp\ipykernel_1852\3516056252.py:5: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  all_dfs.append(pd.read_csv(filename).head(2000))
C:\Users\Sara\AppData\Local\Temp\ipykernel_1852\3516056252.py:5: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  all_dfs.append(pd.read_csv(filename).head(2000))
